In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# input data : 224x224 size
img = torch.rand((1,3,224,224))

In [3]:
# Block은 논문에서 [2,2,2,2], [3,4,6,3]
# 2개씩 묶어서 2 * (3+4+6+3) 으로 생각 + pooling 2회 => ResNet34
# 2* (3,4,6,3) -> shortcut 이랑 그냥 지나간 것을 묶음으로 생각
class Block(nn.Module):
    def __init__(self,in_channels,out_channels,stride=1):
        super().__init__()
        # stride =1, padding=1 -> size변화 없음.
        # stride가 1이 아니라면 -> size 변화 o ex) 128 -> 64 , 64 ->32 등
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2 =  nn.BatchNorm2d(out_channels)
        
        self.relu = nn.ReLU()
        self.shortcut = nn.Sequential()
        
        if stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
    def forward(self,x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [4]:
# resnet 50이상일때 bottle neck구조
# 위는 2개씩 block, 50 이상부터는 3개씩 block
# 1x1 , 3x3, 1x1 로 다시 변환하기 -> 필요한 parameter수를 줄이기
# computation cost 줄일 수 있음 

class ResNet(nn.Module):
    def __init__(self,block,n_blocks,num_classes):
        super(ResNet,self).__init__()
        self.in_channels = 64
        
        # 처음 channel은 3, 이후 7x7x64로 바꾸어주고 block으로 계산됨 
        self.conv1 = nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        # 18일때 n_blocks = [2,2,2,2] 34일때 [3,4,6,3]
        # 처음은 이미 위에서 conv1으로 진행하였으니 stride=1
        self.layer1 = self._make_layer(block,64,n_blocks[0],stride=1)
        self.layer2 = self._make_layer(block,128,n_blocks[1],stride=2)
        self.layer3 = self._make_layer(block,256,n_blocks[2],stride=2)
        self.layer4 = self._make_layer(block,512,n_blocks[3],stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.linear = nn.Linear(512,num_classes)
        
        
    def _make_layer(self,block,out_channels,n_blocks,stride):
        # 처음 stride말고는 size 변화할 일이 없어서 전부 1
        strides = [stride] +[1]*(n_blocks -1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels,out_channels,stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x = self.linear(x)
        return x

In [5]:
test = torch.randn(1,3,224,224) 
y = ResNet(Block,n_blocks=[3,4,6,3],num_classes=1000)
print(y(test).shape)

torch.Size([1, 1000])
